In [6]:
import numpy as np
from keras.utils import to_categorical
from keras.layers import Input, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.optimizers import Adam
from keras.models import Sequential

In [2]:
# labels
with open('PEMS_trainlabels.txt', 'r', encoding='utf-8') as f:
    raw_line = f.readlines()
    train_y = raw_line[0][1:-2].split(' ')

y_train = []
for label in train_y:
    y_train.append(int(label))
y_train = np.array(y_train) - 1
y_train = to_categorical(y_train, 7)

with open('PEMS_testlabels.txt', 'r', encoding='utf-8') as f:
    raw_line = f.readlines()
    test_y = raw_line[0][1:-2].split(' ')

y_test = []
for label in test_y:
    y_test.append(int(label))
y_test = np.array(y_test) - 1
y_test = to_categorical(y_test, 7)

In [3]:
# data
with open('PEMS_train.txt', 'r', encoding='utf-8') as f:
    train_x_lines = f.readlines()

x_train = []
for line in train_x_lines:
    splitted_line = line[1:-1].split(' ')
    values_list = []
    for s in splitted_line:
        try:
            s = float(s)
        except ValueError:
            if ']' not in s:
                s = s.split(';')
                values_list.append(float(s[0]))
                values_list.append(float(s[1]))
            else:
                s.rstrip(']')
                values_list.append(float(s[0]))
        else:
            values_list.append(float(s))
    x_train.append(values_list)

x_train = np.array(x_train)
x_train = x_train.reshape((267, 144, 963))

with open('PEMS_test.txt', 'r', encoding='utf-8') as f:
    test_x_lines = f.readlines()

x_test = []
for line in test_x_lines:
    splitted_line = line[1:-1].split(' ')
    values_list = []
    for s in splitted_line:
        try:
            s = float(s)
        except ValueError:
            if ']' not in s:
                s = s.split(';')
                values_list.append(float(s[0]))
                values_list.append(float(s[1]))
            else:
                s.rstrip(']')
                values_list.append(float(s[0]))
        else:
            values_list.append(float(s))
    x_test.append(values_list)

x_test = np.array(x_test)
x_test = x_test.reshape((173, 144, 963))

In [4]:
print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)
print('x_test.shape: ', x_test.shape)
print('y_test.shape: ', y_test.shape)

x_train.shape:  (267, 144, 963)
y_train.shape:  (267, 7)
x_test.shape:  (173, 144, 963)
y_test.shape:  (173, 7)


In [10]:
model = Sequential()
model.add(Conv1D(64, 5, activation='relu', input_shape=x_train.shape[1:]))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['acc'])
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
9/9 [==============================] - 1s 91ms/step - loss: 1.9744 - acc: 0.1161 - val_loss: 1.9485 - val_acc: 0.1387
Epoch 2/100
9/9 [==============================] - 0s 27ms/step - loss: 1.9038 - acc: 0.1948 - val_loss: 1.9068 - val_acc: 0.1676
Epoch 3/100
9/9 [==============================] - 0s 35ms/step - loss: 1.8393 - acc: 0.2172 - val_loss: 1.8397 - val_acc: 0.1329
Epoch 4/100
9/9 [==============================] - 0s 37ms/step - loss: 1.7278 - acc: 0.2360 - val_loss: 1.7144 - val_acc: 0.2717
Epoch 5/100
9/9 [==============================] - 0s 33ms/step - loss: 1.5682 - acc: 0.4757 - val_loss: 1.6035 - val_acc: 0.3757
Epoch 6/100
9/9 [==============================] - 0s 35ms/step - loss: 1.5043 - acc: 0.4831 - val_loss: 1.4427 - val_acc: 0.4335
Epoch 7/100
9/9 [==============================] - 0s 31ms/step - loss: 1.3122 - acc: 0.4682 - val_loss: 1.3352 - val_acc: 0.4277
Epoch 8/100
9/9 [==============================] - 0s 31ms/step - loss: 1.1994 - acc: 0.48

In [11]:
def model_accuracy(x, y, n=len(x_test)):
    true_cnt = 0
    for _ in range(n):
        rand_indx = np.random.randint(len(x_test))
        pred = np.argmax(model.predict(x_test[rand_indx:rand_indx + 1], verbose=False), axis=1)[0]
        true = np.argmax(y_test[rand_indx])
        if pred == true:
            true_cnt += 1
    print('\nModel accuracy =', true_cnt / n * 100, '%')


model_accuracy(x_test, y_test)


Model accuracy = 82.65895953757226 %


In [12]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


def model_predict(test_indx):
    day_indx = model.predict(np.expand_dims(x_test[test_indx], axis=0), verbose=False)[0].argmax()
    print('Model prediction: ', days[day_indx])
    print('True: ', days[y_test[test_indx].argmax()])


model_predict(26)

Model prediction:  Friday
True:  Friday
